<a href="https://colab.research.google.com/github/anuragal/deep-learning/blob/master/S11/superconvergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cyclic Curve

In [0]:
from cycliclr import plot_cyclic_curve

iterations = 10010
stepsize = 1000
lr_min = 0.08
lr_max = 0.8

plot_cyclic_curve(iterations, stepsize, lr_min, lr_max)

## Model

In [0]:
!pip install albumentations==0.4.5

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip install torchsummary
from torchsummary import summary

from models.S11model import S11Cifar10DNN

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = S11Cifar10DNN().to(device)
summary(model, input_size=(3, 32, 32))

## Run the Model

In [0]:
%matplotlib inline

In [0]:
import torch.nn as nn

from dataloader import ImageData
from models.resnet18 import ResNet18
from datamodel import DataModel
from optimizer import OptimizerFactory

from torch.optim.lr_scheduler import ReduceLROnPlateau

# Loading data and transforming it
img_data = ImageData()

# Choose from "albumentations" or "pytorch". Default is "pytorch"
img_data.load("albumentations")

criterion = nn.CrossEntropyLoss()

# optimizer can be choose from "SGD" & "Adam"
optim_sgd = OptimizerFactory("SGD")
optimizer = optim_sgd.load(params=model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

scheduler = ReduceLROnPlateau(optimizer, mode='min')

# training the dataset and then running test
dm = DataModel(img_data, criterion, optimizer, scheduler, num_of_epochs = 50, cal_misclassified = True)
dm.run_model(model, device)

## Plotting Test & Train Accuracy

In [0]:
dm.plot_test_train_accuracy()

## Plotting Validation Accuracy

In [0]:
dm.plot_loss_accuracy()

## Plotting Misclassified Images

In [0]:
dm.plot_misclassified()

In [0]:
dm.plot_GRADcam(["layer1","layer2","layer3","layer4"])

## Plot LR FInder

In [0]:
from lrfinder import LRFinder

lr_finder = LRFinder(model, optimizer, criterion, device)
lr_finder.range_test(img_data.trainloader, val_loader=img_data.testloader, end_lr=100, num_iter=100, step_mode="exp")
lr_finder.plot(skip_end=0)
